<a href="https://colab.research.google.com/github/xjxgyc/NLP_study/blob/task-20190809/%E4%BD%BF%E7%94%A8cnn%E8%BF%9B%E8%A1%8C%E6%83%85%E6%84%9F%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


在之前的notebook中，我们对情绪分析有了基本的了解。在本notebook中，我们会得到更好的结果。

我们将会使用：

*  打包填充序列
* 预训练的词embedding
* 不同的RNN架构
* 双向RNN
* 多层RNN
* 正则化
* 不同的优化器

这将使我们在test集上达到~84％的准确率。





## 数据准备

和以前一样，我们将设置随机种子，定义`Fields`并获得train / valid / test 的划分。


我们将使用打包填充序列，这将使我们的RNN仅处理我们序列的非填充元素，并且对于任何填充元素，输出将是零张量。为了使用打包填充序列，我们必须告诉RNN实际序列有多长。我们通过为TEXT字段设置`include_lengths = True`来完成此操作。这将导致`batch.text`成为一个元组，第一个元素是我们的句子（已经填充的数值张量），第二个元素是我们句子的实际长度。



In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

加载IMDB数据集

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:10<00:00, 8.13MB/s]



然后从我们的训练集中创建验证集.

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

接下来是使用预先训练的词 embedding。现在，不是让我们的单词embedding随机初始化，而是用这些预先训练的向量初始化它们。我们通过指定我们想要的向量并将其作为参数传递给build_vocab来获取这些向量。 `TorchText`下载向量并将它们与我们词汇表中的正确单词相关联。



在这里，我们将使用“glove.6B.100d”向量。glove是用于计算向量的算法，更多信息可以到[这里](https://nlp.stanford.edu/projects/glove/)查看。6B表示这些向量是在60亿个token上训练的，100d表示这些向量是100维。   



你可以在[这里](https://github.com/pytorch/text/blob/master/torchtext/vocab.py#L113)查看其他可用的向量。


该理论是这些具有相似语义的单词对应的预先训练向量在向量空间中是彼此相近的，例如， "terrible"，"awful"，"dreadful"都比较相近。这为我们的embedding层提供了一个好的初始化，因为它不必从头学习这些关系。





注意：这些向量大约为862MB，因此请注意你是否有网络连接的限制。

默认情况下，`TorchText`会将词汇表中的单词初始化为0，而不会将预训练embedding中的单词初始化为零。我们不希望出现这样的情况，因此将`unk_init`设置为`torch.Tensor.normal_`来随机初始化它们。可以通过使用高斯分布来初始化这些单词。


In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = 'glove.6B.100d',
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [00:53, 16.0MB/s]                           
100%|█████████▉| 399229/400000 [00:15<00:00, 26774.94it/s]

和以前一样，我们创建迭代器，将张量放在GPU上（如果有的话）。

打包填充序列的另一个问题是batch中的所有张量需要按其长度排序。可以通过在迭代器中设置`sort_within_batch = True`来实现。


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
  (train_data, valid_data, test_data),
  batch_size = BATCH_SIZE,
  sort_within_batch = True,
  device = device)

## 创建model


该模型具有剧烈的变化。


### 不同RNN架构

我们将使用称为长短期记忆（LSTM）的RNN架构。为什么LSTM优于标准的RNN？标准RNN会有[梯度消失](https://en.wikipedia.org/wiki/Vanishing_gradient_problem)的问题。 LSTM通过使用一个额外的循环状态来解决这个问题，这个循环状态称为cell，$c$ - 可以被认为是LSTM的“内存” - 并且使用多个门来控制进出内存的信息流。欲了解更多信息，请点击[这里](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)。我们可以简单地将LSTM视为$x_t$，$h_t$和$c_t$的函数，而不仅仅是$x_t$和$h_t$。

$$(h_t, c_t) = \text{LSTM}(x_t, h_t, c_t)$$

因此，使用LSTM的模型看起来像（省略了embedding层）：
![替代文字](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/b1db529eeef1d67a5b0823a63bfcf414b0b569a6/assets/sentiment2.png)

初始单元状态$c_0$与初始隐藏状态一样被初始化为全零的张量。然而，情绪预测仍然仅使用最终隐藏状态，而不是最终单元状态，即：

$\hat{y}=f(h_T)$.

### 双向RNN

双向RNN背后的思想很简单。除了让RNN处理句子中第一个到最后一个的单词（前向RNN）之外，我们还让RNN处理句子中从最后一个到第一个的单词（后向RNN）。在时间步骤$t$，前向RNN正在处理字$x_t$，而后向RNN正在处理字$x_ {T-t + 1}$。



在`PyTorch`中，由前向和后向RNN返回的隐藏状态（和单元状态）张量在单个张量中彼此堆叠。

我们串联来自前向RNN的最后隐藏状态（从句子的最后一个词获得）$h_T^\rightarrow$，以及来自后向RNN的最后隐藏状态（从句子的第一个词获得）$h_T^\leftarrow$， 来进行情感预测，即$\hat{y}=f(h_T^\rightarrow, h_T^\leftarrow)$


下图显示了双向RNN，前向RNN为橙色，后向RNN为绿色，线性层为银色。

![替代文字](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/b1db529eeef1d67a5b0823a63bfcf414b0b569a6/assets/sentiment3.png)





### 多层RNN

多层RNN（也被称为深度RNN）是另一个简单的概念。我们的想法是在初始标准RNN之上添加额外的RNN，其中每个添加RNN是另一层。第一个（底部）RNN在时间步骤$t$输出的隐藏状态将是在时间步骤$t$处对其上方的RNN的输入。然后根据最终（最高）层的最终隐藏状态进行预测。

下图显示了一个多层单向RNN，其中层号用上标表示。另请注意，每层都需要自己的初始隐藏状态，$h_0^L$。


![替代文字](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/b1db529eeef1d67a5b0823a63bfcf414b0b569a6/assets/sentiment4.png)

### 正则化

虽然我们对我们的模型进行了改进，但导致每个model都添加了额外的参数。如果不去过度拟合过多细节，模型中的参数越多，模型过度拟合的概率就越高（记忆训练数据，导致训练误差低，但验证/测试误差高，即在新的，未看过的数据上泛化能力差。）。为了解决这个问题，我们使用正则化。更具体地说，我们使用称为dropout的正则化方法。 Dropout在前向传播期间通过在层中随机丢弃（设置为0）神经元来工作。每个神经元被丢弃的概率由超参数决定，并且每个神经元是否被丢弃是彼此独立的。关于dropout有效的一个理论是，参数丢弃的模型可以被视为“较弱”（较少参数）模型。在模型的参数内所有来自这些“较弱”模型的预测（每个前向传递都有一个预测）被平均在一起。因此，你的模型可以被认为是较弱模型的集成，其中没有一个模型过度参数化，因此不会过度拟合。


### 实现细节

此模型的另一个补充是我们不在学习`<pad>`标记的embedding。这是因为我们想要明确地告诉我们的模型，填充标记与确定句子的情感无关。这意味着`pad`标记的embedding将保持它初始化的内容（我们稍后将其初始化为全零）。我们通过将pad标记的索引作为`padding_idx`参数传递给`nn.Embedding`层来完成此操作。


要使用LSTM而不是标准的RNN，我们使用`nn.LSTM`而不是`nn.RNN`。另请注意，LSTM返回输出以及最终隐藏状态和最终单元状态的元组，而标准RNN仅返回输出和最终隐藏状态。


由于`LSTM`的最终隐藏状态是前向和后向的信息，它们被级联在一起，因此`nn.Linear`层的输入大小是隐藏层大小的两倍。

通过传递`RNN/LSTM`的`num_layers`和`bidirectional`参数的值来实现双向性和添加其他层。


通过初始化`nn.Dropout`层（参数是丢弃每个神经元的概率）来实现Dropout，并在`forward`方法中每个层后面使用dropout。注意：永远不要在输入或输出层上使用dropout（在本例中为text或fc），您只能在中间层上使用dropout。`LSTM`有一个dropout参数，它在某一层的隐藏状态与下一层的隐藏状态之间的连接添加了dropout。


当我们传递句子的长度以便能够使用打包填充序列时，我们必须将第二个参数text_lengths 传递给forward 方法。




在我们将embedding传递给RNN之前，我们需要打包它们，我们使用`nn.utils.rnn.packed_padded_sequence`进行打包。这将导致我们的RNN仅处理序列中的非填充元素。然后RNN将返回`packed_output`（打包序列）以及隐藏和单元状态（两者都是张量）。如果没有进行打包填充序列，隐藏和单元状态是序列中最后一个元素的张量，很可能是填充标记，但是当使用打包填充序列时，它们都来自序列中最后一个非填充元素。

然后，我们使用`nn.utils.rnn.pad_packed_sequence`解压缩输出序列，将其从打包序列转换为张量。填充标记的输出元素将为零张量（每个元素都为零的张量）。通常，我们只会解压缩模型以后需要用到的输出。虽然我们不是这样做的，但我们仍然会解压缩序列，以显示它是如何工作的。


最终隐藏状态， `hidden`的大小是*`[num layers * num directions, batch size, hid dim]`*，它们是有次序的：**[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]**。由于我们需要最终（顶）层前向和后向传播的隐藏状态，我们从第一个维度获取前两个隐藏层，`hidden[-2,:,:]` 和`hidden[-1,:,:]`，在将它们传递到线性层之前（使用dropout之后）级联起来。






In [0]:
import torch.nn as nn

class RNN(nn.Module):
  
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
              bidirectional, dropout, pad_idx):
    
    super().__init__()
    
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
    
    self.rnn = nn.LSTM(embedding_dim,
                       hidden_dim,
                       num_layers = n_layers,
                       bidirectional = bidirectional,
                       dropout = dropout)
    
    self.fc = nn.Linear(hidden_dim * 2, output_dim)
    
    self.dropout = nn.Dropout(dropout)
    
  def forward(self, text, text_lengths):
    
    # text = [sent len, batch size]
    embedded = self.dropout(self.embedding(text))
    
    # embedded = [sent len, batch size, emb dim]
    
    # pack sequence
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
    
    packed_output, (hidden, cell) = self.rnn(packed_embedded)
    
    # unpack sequence
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    
    # output = [sent len, batch size, hid dim * num directions]
    # output over padding tokens are zero tensors
    
    # hidden = [num layers * num directions, batch size, hid dim]
    # cell =  [num layers * num directions, batch size, hid dim]
    
    # concat the fianl forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
    
    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
    
    # hidden = [batch size, hid dim * num directions]
    return self.fc(hidden)

像以前一样，我们将创建一个RNN类的实例，其中包含层数，双向性和丢失概率参数。

为了确保预训练的向量可以加载到模型中，`EMBEDDING_DIM`必须等于先前加载的预训练`GloVe`向量的`EMBEDDING_DIM`。

我们从词汇表中获取`pad` 标记的index，从field 的`pad_token`属性中获取表示`pad`标记的实际字符串，默认情况下为`<pad>`。



In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM, 
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

我们将打印出模型中的参数数量。  

请注意我们的参数几乎是以前的两倍！


In [78]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


最后补充的是将我们之前加载的预训练词embedding复制到模型的embedding层中。 

我们从字段的词汇中检索embedding，并检查它们的大小是否正确，*`[vocab size, embedding dim]`*




In [79]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])



然后，我们用预先训练的embedding替换embedding层的初始权重。



In [80]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0126,  0.6542,  0.9515,  ..., -0.2593,  0.0861, -0.4578],
        [-0.0384,  0.6899,  0.7539,  ...,  0.1326, -0.0184, -0.6284],
        [ 0.1014,  1.2206,  1.1687,  ..., -0.6221,  0.2006,  0.7183]])

当我们在创建词汇表时，我们的`<unk>` 和`<pad>`标记不在预训练的词汇表中，因此使用`unk_init`（一个$\mathcal{N}(0,1)$分布）来初始化它们。最好将它们全部初始化为全零，明确地告诉我们的模型，它们与情绪的确定无关。

我们手动将embedding权重矩阵中的行设置为零来完成此操作。我们已经为这些标记做了填充索引，有了这些索引可以查找到它们对应的行，


In [81]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0126,  0.6542,  0.9515,  ..., -0.2593,  0.0861, -0.4578],
        [-0.0384,  0.6899,  0.7539,  ...,  0.1326, -0.0184, -0.6284],
        [ 0.1014,  1.2206,  1.1687,  ..., -0.6221,  0.2006,  0.7183]])


我们现在可以看到embedding权重矩阵的前两行已设置为零。在整个训练的过程中，传递给embedding层`padding_idx`的pad 标记embedding一直为0，
但是`<UNK>`标记的embedding将被学习到。



### 训练model

现在开始训练model。

我们在这里做的唯一改变是将优化器从`SGD`更改为`Adam`。 `SGD`以相同的学习率更新所有参数，选择学习率需要很多技巧。 `Adam`调整每个参数的学习率，给频繁更新的参数以低的学习率和不频繁更新的以高的学习率。有关`Adam`（和其他优化器）的更多信息，请访问[此处](http://ruder.io/optimizing-gradient-descent/index.html)。

 
 要将`SGD`更改为`Adam`，我们只需将`optim.SGD`更改为`optim.Adam`，还要注意我们不必为`Adam`提供初始学习率，因为`PyTorch`指定了一个敏感的默认初始学习率。



In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

用于训练模型的其余步骤保持不变。

我们定义了评估标准并将模型和评估标准放在GPU上（如果可用）......



In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

我们实现了一个计算准确率的函数......




In [0]:
def binary_accuracy(preds, y):
  """
  Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
  """
  
  # round predictions to the closest integer
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float() ## convert into float for division
  acc = correct.sum() / len(correct)
  return acc

我们定义一个训练模型的函数。

设置`include_lengths = True`，`batch.text`现在就是一个元组，第一个元素是数值张量，第二个元素是每个序列的实际长度。在将它们传递给模型之前，我们将它们分为变量text和text_lengths。

注意：由于我们现在正在使用dropout，我们必须使用`model.train()`来确保在训练时“开启”dropout。



In [0]:
def train(self, iterator, optimizer, criterion):
  
  epoch_loss = 0
  epoch_acc = 0
  
  model.train()
  
  for batch in iterator:
    
    optimizer.zero_grad()
    
    text, text_lengths = batch.text
    
    predictions = model(text, text_lengths).squeeze(1)
    
    loss = criterion(predictions, batch.label)
    
    acc = binary_accuracy(predictions, batch.label)
    
    loss.backward()
    
    optimizer.step()
    
    epoch_loss += loss.item()
    epoch_acc += acc.item()
    
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

然后我们定义一个测试模型的函数，再次记住要分离`batch.text`。

注意：由于我们现在使用dropout，我们必须记住使用`model.eval()`来确保在评估时“关闭”dropout。




In [0]:
def evaluate(model, iterator, criterion):
  
  epoch_loss = 0
  epoch_acc = 0
  
  model.eval()
  
  with torch.no_grad():
    
    for batch in iterator:
      
      text, text_lengths = batch.text
      
      predictions = model(text, text_lengths).squeeze(1)
      
      loss = criterion(predictions, batch.label)
      
      acc = binary_accuracy(predictions, batch.label)
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()
      
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

我们也定义一个函数，让它告知我们一个epoch 花费多长时间。

In [0]:
import time

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - elapsed_mins * 60)
  return elapsed_mins, elapsed_secs

最后，我们训练我们的模型...

In [89]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  
  start_time = time.time()
  
  train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
  
  end_time = time.time()
  
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'tut2-model.pt')
    
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
  print(f'\t .Val Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01 | Epoch Time: 0m 44s
	Train Loss: 0.313 | Train Acc: 87.24%
	 .Val Loss: 0.479 |  Val. Acc: 78.99%
Epoch: 02 | Epoch Time: 0m 44s
	Train Loss: 0.266 | Train Acc: 89.51%
	 .Val Loss: 0.300 |  Val. Acc: 88.02%
Epoch: 03 | Epoch Time: 0m 44s
	Train Loss: 0.253 | Train Acc: 90.15%
	 .Val Loss: 0.280 |  Val. Acc: 88.28%
Epoch: 04 | Epoch Time: 0m 44s
	Train Loss: 0.232 | Train Acc: 90.92%
	 .Val Loss: 0.367 |  Val. Acc: 86.77%
Epoch: 05 | Epoch Time: 0m 44s
	Train Loss: 0.193 | Train Acc: 92.79%
	 .Val Loss: 0.322 |  Val. Acc: 87.55%


...获得我们新的，高的测试精度！

In [90]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Test Loss: 0.292 | Test Acc: 87.90%


## 用户输入

现在，我们可以使用模型来预测我们给出的任何句子的情绪。由于该模型是在电影评论数据集上训练的，所给出的句子也应该是电影评论。

使用模型进行推理时，它应始终处于评估模式。如果按照本教程逐步进行，那么它应该已经处于评估模式（对测试集进行评估），但是我们明确地进行设置，以避免出错。



我们的`predict_sentiment`函数做的事情：

> 将模型设置为评估模式   
>
> 对句子进行分词，即将原始字符串分割为标记列表  
>
> 使用我们的词汇表将标记转换为整数来建立索引
>
> 得到序列的长度
>
> 将Python列表的索引转换为PyTorch张量  
>
> 使用`unsqueezeing`添加batch维度  
>
> 将长度转换为张量   
>
> 使用`sigmoid`函数将输出预测值压缩到0~1之间的实数
>
> 使用`item()`方法将单个值的张量转换为整数


我们期望带有负面情绪的评论会返回接近0的值，而正面评价则会返回接近1的值。








In [0]:
import spacy

nlp = spacy.load('en')

def predict_sentiment(model, sentence):
  model.eval()
  tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
  indexed = [TEXT.vocab.stoi[t] for t in tokenized]
  length = [len(indexed)]
  tensor = torch.LongTensor(indexed).to(device)
  tensor = tensor.unsqueeze(1)
  length_tensor = torch.LongTensor(length)
  prediction = torch.sigmoid(model(tensor, length_tensor))
  return prediction.item()

一个负面评论的例子：



In [95]:
predict_sentiment(model, "This film is terrible")


0.006773746572434902

一个正面评论的例子：



In [96]:
predict_sentiment(model, "This film is greate")

0.8507424592971802

## 下一步

现在我们为电影评论建立了一个不错的情绪分析模型！在下一个notebook中，我们将实现一个模型，该模型可以获得同样的精度，参数更少，训练得更快。

